In [1]:
from pythreejs import *
import numpy as np
from IPython.display import display
from ipywidgets import HTML, Text
from traitlets import link, dlink

In [2]:
ball = Mesh(geometry=SphereGeometry(radius=1), 
                                    material=LambertMaterial(color='red'),
                                    position=[2, 1, 0])

scene = Scene(children=[ball, AmbientLight(color='#777777')])

c = PerspectiveCamera(position=[0, 5, 5], up=[0, 0, 1],
                      children=[DirectionalLight(color='white', 
                                                 position=[3, 5, 1], 
                                                 intensity=0.5)])
renderer = Renderer(camera=c, 
                    scene=scene, 
                    controls=[OrbitControls(controlling=c)])
display(renderer)

In [3]:
# On windows, linewidth of the material has no effect
size = 4
linesgeom = PlainGeometry(vertices=[[0, 0, 0],
                                    [size, 0, 0],
                                    [0, 0, 0],
                                    [0, size, 0],
                                    [0, 0, 0],
                                    [0, 0, size]],
                          colors = ['red', 'red', 'green', 'green', 'white', 'orange'])
lines = Line(geometry=linesgeom, 
             material=LineBasicMaterial(linewidth=5, vertexColors='VertexColors'), 
             type='LinePieces')
scene = Scene(children=[lines, DirectionalLight(color='#ccaabb', position=[0,10,0]),AmbientLight(color='#cccccc')])
c = PerspectiveCamera(position=[0, 10, 10])
renderer = Renderer(camera=c, scene = scene, controls=[OrbitControls(controlling=c)])
display(renderer)

In [4]:
import ase
import ase.io